# Initialisation

In [36]:
!pip3 install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Imports

In [37]:
import torch
import logging
import requests
from tqdm import tqdm
import pandas as pd
import os

logging.basicConfig(level=logging.INFO)# OPTIONAL
print(f"PyTorch version: {torch.__version__}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 2.5.1
Using device: mps


# Global Variables

In [38]:
#Data Source
OUTPUT_TEMPLATE = 'output_template/'
TEMPLATES_COMPLETE_PATH = OUTPUT_TEMPLATE + 'template_complete.csv'
DATA_SOURCE = 'dataset_source/'
OUTPUT_PREDICTION = 'output_prediction/'

In [ ]:
url = "http://localhost:11434/api/generate"

class generateSentences():
    def __init__(self):
            self.templateFile = pd.read_csv(TEMPLATES_COMPLETE_PATH, sep=";")
            dicSentences = {
                "Template": [],
                "Generated": []
            }
            print("Generating sentences...")
            for index,row in tqdm(self.templateFile[:20].iterrows(), total=self.templateFile.shape[0], desc='Generating ', unit=' sentences'):
                sentence = row.loc['template']
                #Other propts to try:
                #prompt = f"Complete the follwing sentence and provide the response {sentence}"
                #prompt = "Complete the follwing sentence (return only one response): {0}".format(sentence)
                
                prompt = f"Complete the following sentence: `{sentence}` Provide only the additional words necessary to complete the sentence as output, without repeating the initial part or adding any explanations."
                response = self.llama3(prompt)
                dicSentences["Template"].append(sentence)
                dicSentences["Generated"].append(sentence + response)
                #print(str(index) +"-"+ sentence + response)
            df = pd.DataFrame.from_dict(dicSentences)    
            #display(df)
            print("Sentences generated!")            
            os.makedirs(OUTPUT_PREDICTION, exist_ok=True)
            df.to_csv(OUTPUT_PREDICTION+'gemma.csv', sep=';', index=False)
            print("File generated!!")
    
    def llama3 (self, prompt):
        data = {
            "model": "llama3",
            "prompt": prompt,
            "messages": [
                {
                "role": "user",
                "content": prompt
                }
            ],   
            "stream": False
        }
        
        headers = {
            "Content-Type": 'application/json'
        }

        response = requests.post (url, headers=headers, json=data)
        #return (response.json()['choices'][0]['message']['content'])
        return(response.json()['response'])
    

In [ ]:
generateSentences()

Generating sentences...


Generating :   1%|          | 66/6572 [00:56<1:19:12,  1.37 sentences/s]